# Creating a basic chat experience with context variables

In [1]:
import os

In [2]:
import semantic_kernel as sk
kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(
        deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    )
)

Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'chat_completion': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x000001AD683EF040>}, chat_services={'chat_completion': <function Kernel.add_chat_service.<locals>.<lambda> at 0x000001AD47D3B430>}, text_embedding_generation_services={}, default_text_completion_service='chat_completion', default_chat_service='chat_completion', default_text_embedding_generation_service=None, retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

In [3]:
sk_prompt = """
Chatbot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$input}}
ChatBot:"""

Register your semantic function

In [4]:
chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.8,
    top_p=0.5
)

In [5]:
context = kernel.create_new_context()
context["history"] = ""

Chat with Bot

In [6]:
context["user_input"] = "Hi, I am looking for a book suggestions"
bot_answer = await chat_function.invoke(context=context)
print(bot_answer)

Variable `$history` not found
Variable `$input` not found


Hello! How can I assist you today?


In [7]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context['history'])


User: Hi, I am looking for a book suggestions
ChatBot: Hello! How can I assist you today?



In [9]:
context["user_input"] = "Require Machine learning books"
bot_answer = await chat_function.invoke(context=context)
print(bot_answer)

I apologize for the repetition. I am an AI chatbot and I am here to assist you with any questions or concerns you may have. How can I help you today?


In [10]:
async def chat(input_text: str) -> None:
    # save the message in context variable
    print(f"User: {input_text}")
    context['user_input'] = input_text

    # process the user message and get an answer
    answer = await chat_function.invoke(context=context)

    print(f"ChatBot: {answer}")

    # append the new interation to the chat history
    context['history'] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [11]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Oh, I see! Well, I'm actually looking for some book suggestions. Can you recommend any good books for me to read?


In [12]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: Sure! How about "The Greeks" by H.D.F. Kitto or "The Iliad" by Homer? Both are great options for learning more about Greece's history and culture.


In [13]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: Great choices! "The Greeks" by H.D.F. Kitto is a comprehensive overview of ancient Greek civilization, covering everything from politics and philosophy to art and literature. "The Iliad" by Homer is an epic poem that tells the story of the Trojan War and is considered one of the greatest works of Western literature. Both books are sure to provide you with a deeper understanding of Greece's rich history and culture.


In [14]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: I'm glad you like the suggestions! Is there anything else I can help you with today?


After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [15]:
print(context["history"])


User: Hi, I am looking for a book suggestions
ChatBot: Hello! How can I assist you today?

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Oh, I see! Well, I'm actually looking for some book suggestions. Can you recommend any good books for me to read?

User: that sounds interesting, what is it about?
ChatBot: Sure! How about "The Greeks" by H.D.F. Kitto or "The Iliad" by Homer? Both are great options for learning more about Greece's history and culture.

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: Great choices! "The Greeks" by H.D.F. Kitto is a comprehensive overview of ancient Greek civilization, covering everything from politics and philosophy to art and literature. "The Iliad" by Homer is an epic poem that tells the story of the Trojan War and is considered one of the greatest works of Western literature. Both books are sure to provide you with a deeper understanding of Greece's ric